# Эксперименты с автоэнкодером

## Подготовка данных

In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt

from torch import nn
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter

from src.utils import plot_reconstructed, grid_plot

/Users/citizen2/Documents/miltask/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = CIFAR10('../data/', train=True, download=True)

mean = data.data.mean(axis=(0,1,2))/255
std = data.data.std(axis=(0,1,2))/255
print(f'mean: {mean}')
print(f'std: {std}')

Files already downloaded and verified
mean: [0.49139968 0.48215841 0.44653091]
std: [0.24703223 0.24348513 0.26158784]


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(mean, std)
])


data = CIFAR10('../data/', train=True, download=True, transform=transform)
data_loader = torch.utils.data.DataLoader(dataset=data, batch_size=50, shuffle=True)

Files already downloaded and verified


# Обучение автоэнкодера

In [99]:
def train(net):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)


    num_epochs = 1
    tb = SummaryWriter()

    step = 1
    for epoch in range(num_epochs):
        for batch, _ in data_loader:
            reconstructed = net(batch)

            loss = criterion(reconstructed, batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            tb.add_scalar("Loss/train", loss, step)
            if step%20 == 0:
                    grid = grid_plot(batch, reconstructed)
                    tb.add_image('Original vs Reconstructed', grid, step)

            step += 1

    tb.add_graph(net, batch)
    tb.close()

In [100]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),

            nn.Conv2d(64, 4, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU()
            )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(4, 64, 3, stride=2, padding=1, output_padding=1),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(64, 3, 3, stride=1, padding=1),
            nn.LeakyReLU(),
        )

    def forward(self, x):
        coded = self.encoder(x)
        decoded = self.decoder(coded)

        return decoded

net = Autoencoder()
train(net)

In [104]:
from torchinfo import summary

summary(Autoencoder(), input_size=(1, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
Autoencoder                              [1, 3, 32, 32]            --
├─Sequential: 1-1                        [1, 4, 16, 16]            --
│    └─Conv2d: 2-1                       [1, 64, 32, 32]           1,792
│    └─LeakyReLU: 2-2                    [1, 64, 32, 32]           --
│    └─Conv2d: 2-3                       [1, 4, 16, 16]            2,308
│    └─LeakyReLU: 2-4                    [1, 4, 16, 16]            --
├─Sequential: 1-2                        [1, 3, 32, 32]            --
│    └─ConvTranspose2d: 2-5              [1, 64, 32, 32]           2,368
│    └─LeakyReLU: 2-6                    [1, 64, 32, 32]           --
│    └─ConvTranspose2d: 2-7              [1, 3, 32, 32]            1,731
│    └─LeakyReLU: 2-8                    [1, 3, 32, 32]            --
Total params: 8,199
Trainable params: 8,199
Non-trainable params: 0
Total mult-adds (M): 6.62
Input size (MB): 0.01
Forward/backward pass size 

## Оценка качества



Сохраняем понравившийся вариант

In [ ]:
torch.save(net.state_dict(), '../outputs/clf_model.pth')